In this notebook we compute the statistics from the predictions. The reason for using R instead of python in this occasion is to able to use the BayesFactor package

In [7]:
require(BayesFactor)
require(boot)

In [8]:
r2_score<-function(y_pred, y_true){
    ss_res<- sum((y_pred-y_true)**2)
    
    mu_true = mean(y_true)
    
    ss_tot<-sum((y_true-mu_true)**2)
    
    r2<-1 - ss_res/ss_tot
    r2
}

mean_absolute_error<-function(y_pred, y_true){
    mean(abs(y_pred-y_true))
}

mean_square_error<-function(y_pred, y_true){
    mean((y_pred-y_true)**2)
}

In [9]:
results_dir<-"../results/prediction"

In [10]:
RANDOM_STATE<-0

In [11]:
res_df <- data.frame(target=character(),
                     contrast=character(), 
                     r=numeric(),
                     r_l_95CI = numeric(),
                     r_u_95CI = numeric(),
                     pval = numeric(),
                     #MSE=numeric(),
                     MAE=numeric(),
                     R2=numeric(), 
                     BF10 = numeric(),
                     BF01 = numeric(),
                     stringsAsFactors=FALSE) 


set.seed(RANDOM_STATE)
row_ix<-1

cases<-list(c("look_neg_look_neut", "chg_LNeg_LNur"),
            c("reg_neg_look_neg", "chg_RNeg_LNeg")
           )

for(ii in c(1,2)){
    contrast_id <- cases[[ii]][1]
    targ_case <- cases[[ii]][2]
    
    res_case<-c(targ_case, contrast_id)

    y_results_df<-read.csv(file.path(results_dir, targ_case, contrast_id, "y_preds.csv"))

    y_pred<-y_results_df[,"y_pred"]
    y_true<-y_results_df[,"y_true"]

    r2<-r2_score(y_pred, y_true)
    mse<-mean_square_error(y_pred, y_true)
    mae<-mean_absolute_error(y_pred, y_true)

    corrs.res<-cor.test(y_pred, y_true, alternative="greater")
    r<-corrs.res$estimate
    b_object <- boot(y_results_df, 
               statistic = function(data, i){cor(data[i, "y_pred"], data[i, "y_true"], method='pearson')},
               R = 1000)
    r_boots<-boot.ci(b_object, type = "perc")

    pv<-corrs.res$p.value

    r_l_95CI <- r_boots$perc[4]
    r_u_95CI <- r_boots$perc[5]

    bf_10<-as.vector(correlationBF( y_true, y_pred, nullInterval = c(0,1))[1])
    bf_01<-1/bf_10

    res_case<-c(res_case, round(c(r, r_l_95CI, r_u_95CI,  pv, mae, r2, bf_10, bf_01), 3))
    res_df[row_ix, ]<- res_case
    row_ix =row_ix + 1

    
}

res_df

,target,contrast,r,r_l_95CI,r_u_95CI,pval,MAE,R2,BF10,BF01
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,chg_LNeg_LNur,look_neg_look_neut,0.143,-0.027,0.308,0.043,0.52,0.015,1.521,0.658
2,chg_RNeg_LNeg,reg_neg_look_neg,0.457,0.306,0.593,0,0.418,0.209,3819342.53,0


In [5]:
res_df <- data.frame(target=character(),
                     contrast=character(), 
                     r=numeric(),
                     r_l_95CI = numeric(),
                     r_u_95CI = numeric(),
                     pval = numeric(),
                     #MSE=numeric(),
                     MAE=numeric(),
                     R2=numeric(), 
                     BF10 = numeric(),
                     BF01 = numeric(),
                     stringsAsFactors=FALSE) 


set.seed(RANDOM_STATE)
row_ix<-1

cases<-list(c("look_neg_look_neut", "chg_LNeg_LNur"),
            c("reg_neg_look_neg", "chg_RNeg_LNeg")
           )

for(ii in c(1,2)){
    contrast_id <- cases[[ii]][1]
    targ_case <- cases[[ii]][2]
    
    res_case<-c(targ_case, contrast_id)

    y_results_df<-read.csv(file.path(results_dir, targ_case, contrast_id, "y_preds.csv"))

    y_pred<-y_results_df[,"y_pred"]
    y_true<-y_results_df[,"y_true"]

    r2<-r2_score(y_pred, y_true)
    mse<-mean_square_error(y_pred, y_true)
    mae<-mean_absolute_error(y_pred, y_true)

    corrs.res<-cor.test(y_pred, y_true, alternative="greater")
    r<-corrs.res$estimate
    b_object <- boot(y_results_df, 
               statistic = function(data, i){cor(data[i, "y_pred"], data[i, "y_true"], method='pearson')},
               R = 1000)
    r_boots<-boot.ci(b_object, type = "perc")

    pv<-corrs.res$p.value

    r_l_95CI <- r_boots$perc[4]
    r_u_95CI <- r_boots$perc[5]

    bf_10<-as.vector(correlationBF( y_true, y_pred, nullInterval = c(0,1))[1])
    bf_01<-1/bf_10

    res_case<-c(res_case, round(c(r, r_l_95CI, r_u_95CI,  pv, mae, r2, bf_10, bf_01), 3))
    res_df[row_ix, ]<- res_case
    row_ix =row_ix + 1

    
}

res_df

,target,contrast,r,r_l_95CI,r_u_95CI,pval,MAE,R2,BF10,BF01
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,chg_LNeg_LNur,look_neg_look_neut,0.143,-0.027,0.308,0.043,0.52,0.015,1.515,0.66
2,chg_RNeg_LNeg,reg_neg_look_neg,0.457,0.306,0.593,0,0.418,0.209,3817080.417,0
